In [1]:

import os
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

import matplotlib.dates
from datetime import datetime

import plotly.express as px
from difflib import SequenceMatcher

In [2]:
path_1 = 'data/src/Reporte_Reclamos 2018.xls'
path_2 = 'data/src/Reporte_Reclamos 2019.xls'
path_3 = 'data/src/Reporte_Reclamos 2020.xlsx'
path_4 = 'data/src/Reporte_Reclamos 2021.xlsx'

paths = [path_1, path_2, path_3, path_4]

data_reclamos = pd.DataFrame()
for p in paths:
    data_temp = pd.read_excel(p, index_col=0, header=0)
    data_reclamos = pd.concat([data_reclamos,data_temp])

In [3]:
data_reclamos.head()

,Fase1_presen,TEMPORAL,CALIFFINAL,Fase4_generac,Fase5_revisac,Fase6_enviac,Fase7_firma_ACUSE,N° Reclamo,Nombre,N° Docum.,...,Nombre Usuario,Descripción Fase,Canal Ingreso,Medio Respuesta,Reclamo/requerim,Hace carta,Correo,Cta. Ahorro,Descripcion,Motivo.1
N°,,,,,,,,,,,,,,,,,,,,,
1,31/12/2018,17/01/2019,17/01/2019,21/01/2019,21/01/2019,17/01/2019,17/01/2019,002718-7736,ADAN HOLGUIN SOLIS,42672067,...,ANGLAS/RAMIREZ/ELSA/MARIELA,Firma,PÁGINA WEB BN,Correo Electrónico,RECLAMO,GGMA,adansmith1984@hotmail.com,4052857215.0,"EN FECHA 20 DE DICIEMBRE, EN HORAS DE LA TARDE...",NaN
2,31/12/2018,03/01/2019,03/01/2019,03/01/2019,03/01/2019,03/01/2019,03/01/2019,067618-34,ADANAQUE TIZON MARIA LILIANA,45056104,...,MARCHAN/GARCIA/JORGE,Firma,AGENCIA BN,Carta en su domicilio,RECLAMO,TCML,NaN,4676106532.0,RETIRO Y NO DIO EFECTIVO NI BOUCHER,NaN
3,31/12/2018,16/01/2019,17/01/2019,17/01/2019,17/01/2019,17/01/2019,17/01/2019,002718-7726,ADEMIR CHAVEZ SANCHEZ,41311452,...,WONG/MOTTA/ALEXANDRA,Firma,PÁGINA WEB BN,Carta en su domicilio,RECLAMO,GBNL,CONTACTAME.OK@GMAIL.COM,NaN,EL DIA 29 DE DICIEMBRE A LAS 18:00 EFECTUE UN ...,NaN
4,31/12/2018,09/01/2019,09/01/2019,09/01/2019,09/01/2019,09/01/2019,09/01/2019,315118-10281,AGUERO OLIVAS JOHN RICHARD,40383052,...,VALENCIA/MEDINA/TANIA IBET,Firma,MESA DE AYUDA (TELEFÓNO),Correo Electrónico,RECLAMO,RRBM,richaragol@gmail.com,4044548211.0,TITULAR MANIFIESTA QUE SE ACERCO A CAJERO DEL ...,NaN
5,31/12/2018,17/01/2019,17/01/2019,17/01/2019,17/01/2019,17/01/2019,17/01/2019,034618-9,AGUILAR VEGA JULIO EDUARDO,45451586,...,HERRERA/MEDINA/NESTOR,Firma,AGENCIA BN,Correo Electrónico,RECLAMO,SJAR,aguilar_88_2013@hotmail.com,4346109217.0,SOLICITA DEVOLUCION DE DINERO QUE NO FUE DISPE...,NaN


In [4]:
# data_reclamos['Cod Motivo'] = data_reclamos['Cod Motivo'].fillna(0)
# data_reclamos['Cod Motivo'] = data_reclamos['Cod Motivo'].astype(int)

### Correccion de nombre de ciudades

Se incorporo un archivo de centros poblados para facilitarnos la correcion

In [5]:
path_geopolitico = 'data/src/BaseCentrosPoblados.csv'
data_geopolitico = pd.read_csv(path_geopolitico, delimiter=';')
data_geopolitico = data_geopolitico.groupby(['DEPARTAMEN','PROVINCIA','DISTRITO']).agg({ 'POB': "sum"})
data_geopolitico = data_geopolitico.sort_values(by=['DEPARTAMEN','PROVINCIA','DISTRITO']).reset_index()

# correccion de departamentos
def similars(text, lista):
    max_val = 0
    resp = ''
    for e in lista:
        s = SequenceMatcher(None, str(text), e)
        if s.ratio() > max_val:
            max_val = s.ratio()
            resp = e
        if max_val == 1:
            return resp
    if max_val >= 0.8:
        return resp
    return text

def similars2(text, lista):
    max_val = 0
    resp = ''
    for e in lista:
        s = SequenceMatcher(None, str(text), e)
        if s.ratio() > max_val:
            max_val = s.ratio()
            resp = e
        if max_val == 1:
            return max_val
    return max_val

lista = list(np.unique(data_geopolitico['DEPARTAMEN']))
data_reclamos['DEPARTAMEN'] = data_reclamos['Departam. Domic.'].apply(similars, args=(lista,))
#data_reclamos['DEPARTAMEN_VAL'] = data_reclamos['Departam. Domic.'].apply(similars2, args=(lista,))

lista = list(np.unique(data_geopolitico[data_geopolitico['DEPARTAMEN']=='LIMA']['PROVINCIA']))
data_reclamos['PROVINCIA'] = data_reclamos.apply(lambda x: similars(x['Provincia Domic.'], lista) if x['Departam. Domic.']=='LIMA' else x['Provincia Domic.'],axis=1)
#data_reclamos['PROVINCIA_VAL'] = data_reclamos.apply(lambda x: similars2(x['Provincia Domic.'], lista) if x['Departam. Domic.']=='LIMA' else 0,axis=1)

lista = list(np.unique(data_geopolitico[(data_geopolitico['DEPARTAMEN']=='LIMA') & (data_geopolitico['PROVINCIA']=='LIMA')]['DISTRITO']))
data_reclamos['DISTRITO'] = data_reclamos.apply(lambda x: similars(x['Distrito Domic.'], lista)  if x['Departam. Domic.']=='LIMA' and x['Provincia Domic.']=='LIMA' else x['Distrito Domic.'],axis=1)
data_reclamos['DISTRITO_VAL'] = data_reclamos.apply(lambda x: similars2(x['Distrito Domic.'], lista) if x['Departam. Domic.']=='LIMA' and x['Provincia Domic.']=='LIMA' else 0,axis=1)

data_reclamos['IS_LIMA'] = data_reclamos.apply(lambda x: True if x['DISTRITO_VAL']>=0.8 else False, axis=1)

del data_reclamos['DISTRITO_VAL']
del data_reclamos['Departam. Domic.']
del data_reclamos['Provincia Domic.']
del data_reclamos['Distrito Domic.']

### Correccion de fechas
Se incorporo un archivo de centros poblados para facilitarnos la correcion

In [6]:
data_reclamos['fecha_fase1'] = pd.to_datetime(data_reclamos['Fase1_presen'], format='%d/%m/%Y')
data_reclamos['fecha_fase2'] = pd.to_datetime(data_reclamos['TEMPORAL'], format='%d/%m/%Y')
data_reclamos['fecha_fase3'] = pd.to_datetime(data_reclamos['CALIFFINAL'], format='%d/%m/%Y')
data_reclamos['fecha_fase4'] = pd.to_datetime(data_reclamos['Fase4_generac'], format='%d/%m/%Y')
data_reclamos['fecha_fase5'] = pd.to_datetime(data_reclamos['Fase5_revisac'], format='%d/%m/%Y')
data_reclamos['fecha_fase6'] = pd.to_datetime(data_reclamos['Fase6_enviac'], format='%d/%m/%Y')
data_reclamos['fecha_fase7'] = pd.to_datetime(data_reclamos['Fase7_firma_ACUSE'], format='%d/%m/%Y')

del data_reclamos['Fase1_presen']
del data_reclamos['TEMPORAL']
del data_reclamos['CALIFFINAL']
del data_reclamos['Fase4_generac']
del data_reclamos['Fase5_revisac']
del data_reclamos['Fase6_enviac']
del data_reclamos['Fase7_firma_ACUSE']

### Nan in codigos de servicio y motivo

In [7]:
data_reclamos['Cod. Servicio'] = data_reclamos['Cod. Servicio'].fillna(-1).astype(int).astype(str)
data_reclamos['Cod Motivo'] = data_reclamos['Cod Motivo'].fillna(-1).astype(int).astype(str)


### Renombrando columnas

In [8]:
data_reclamos.rename(columns={
    'N° Reclamo': 'Numero reclamo',
    'Nombre': 'Nombre',
    'N° Docum.': 'Numero documento',
    'Cod. Servicio': 'Cod servicio',
    'Servicio': 'Servicio',
    'Cod Motivo': 'Cod motivo',
    'Motivo': 'Motivo',
    'Cod. Atm': 'Cod ATM',
    'Det. AtmError': 'ATM error',
    'N° Cajero': 'Numero cajero',
    'Monto': 'Monto',
    'Operac.': 'Fecha operacion',
    'Present.': 'Fecha presentacion',
    'Resul.': 'Fecha resultado',
    'Recl.': 'Estado reclamo',
    'Cod. Oficina': 'Cod oficina',
    'Det. Oficina': 'Det oficina',
    'Regional': 'Cod region',
    'Dependencia': 'Dependencia',
    'Cod. Deriv': 'Cod dependencia derivada',
    'Última Derivació': 'Ultima derivacion',
    'N° Memo Carta Resp.': 'Numero carta respuesta',
    'Dirección Domic.': 'Direccion domicilio',
    'Fase del Reclamo': 'Fase reclamo',
    'Proceso': 'error_1',
    'Días de Demora': 'error_2',
    ' de Demora': 'error_3',
    'Respon': 'error_4',
    'Responsable': 'error_5',
    'Macroregión': 'error_6',
    'Resp.': 'error_7',
    'División': 'error_8',
    'División Responsable': 'error_9',
    'Cod. Dept': 'error_10',
    'Responsable.1': 'error_11',
    'Mes Presentacion': 'Mes presentacion',
    'Días Reclamo': 'Dias reclamo',
    'Año Presentación': 'Anio presentacion',
    'Semáforo': 'error_12',
    'Situación': 'error_13',
    'Cod. Usuario': 'Cod usuario',
    'Nombre Usuario': 'Usuario',
    'Descripción Fase': 'error_14',
    'Canal Ingreso': 'Canal ingreso',
    'Medio Respuesta': 'Medio respuesta',
    'Reclamo/requerim': 'Categoria',
    'Hace carta': 'Hace carta',
    'Correo': 'Correo electronico',
    'Cta. Ahorro': 'Cuenta ahorro',
    'Descripcion': 'Descripcion',
    'DEPARTAMEN': 'Departamento',
    'PROVINCIA': 'Provincia',
    'DISTRITO': 'Distrito',
    'IS_LIMA': 'Es Lima',
    'fecha_fase1': 'fecha fase 1',
    'fecha_fase2': 'fecha fase 2',
    'fecha_fase3': 'fecha fase 3',
    'fecha_fase4': 'fecha fase 4',
    'fecha_fase5': 'fecha fase 5',
    'fecha_fase6': 'fecha fase 6',
    'fecha_fase7': 'fecha fase 7'
}, inplace = True)

### Eliminando columnas erroneas
Nos basamos en el documento compartido por Negocio:
 - Descripción de campos AISR

In [9]:
columns = data_reclamos.columns
news_columns = []
for i in columns:
    if 'error_' in i:
        pass
    else:
        news_columns.append(i)

data_reclamos = data_reclamos[news_columns]

### Eliminando columnas que seran remplazadas por los catalogos

In [10]:
del data_reclamos['Motivo']
del data_reclamos['Servicio']


### Guardando archivo (PRTK)


In [11]:
file_name = 'data/pre_processed_data.csv'
data_reclamos.to_pickle(file_name)

In [12]:
data_reclamos.head()

,Numero reclamo,Nombre,Numero documento,Cod servicio,Cod motivo,Cod ATM,ATM error,Numero cajero,Monto,Fecha operacion,...,Provincia,Distrito,Es Lima,fecha fase 1,fecha fase 2,fecha fase 3,fecha fase 4,fecha fase 5,fecha fase 6,fecha fase 7
N°,,,,,,,,,,,,,,,,,,,,,
1,002718-7736,ADAN HOLGUIN SOLIS,42672067,2200,280800,5.0,(cajero multired) COD.82-ERROR EN EL ATM,NaN,NaN,20/12/2018,...,LIMA,LOS OLIVOS,True,2018-12-31,2019-01-17,2019-01-17,2019-01-21,2019-01-21,2019-01-17,2019-01-17
2,067618-34,ADANAQUE TIZON MARIA LILIANA,45056104,2200,280800,5.0,(cajero multired) COD.82-ERROR EN EL ATM,774.0,200,31/12/2018,...,PIURA,TAMBOGRANDE,False,2018-12-31,2019-01-03,2019-01-03,2019-01-03,2019-01-03,2019-01-03,2019-01-03
3,002718-7726,ADEMIR CHAVEZ SANCHEZ,41311452,2200,280800,99.0,(cajero multired) OTROS,NaN,20,29/12/2018,...,LIMA,VILLA MARIA DEL TRIUNFO,True,2018-12-31,2019-01-16,2019-01-17,2019-01-17,2019-01-17,2019-01-17,2019-01-17
4,315118-10281,AGUERO OLIVAS JOHN RICHARD,40383052,2200,280800,99.0,(cajero multired) OTROS,1338.0,2580,31/12/2018,...,HUANUCO,HUANUCO,False,2018-12-31,2019-01-09,2019-01-09,2019-01-09,2019-01-09,2019-01-09,2019-01-09
5,034618-9,AGUILAR VEGA JULIO EDUARDO,45451586,2200,280800,5.0,(cajero multired) COD.82-ERROR EN EL ATM,789.0,400,23/12/2018,...,POMABAMBA,POMABAMBA,False,2018-12-31,2019-01-17,2019-01-17,2019-01-17,2019-01-17,2019-01-17,2019-01-17
